# Imports

In [4]:
import bz2
import os.path as path

# Constants

In [27]:
DATA_DIR = '../data'

# Functions

In [28]:
# Validate all compressed files are present data directory
def validate_dump(data_dir, date='2021-10-20') -> bool:
    # Check for existence of data directory
    if not path.exists(dump_dir:=path.join(data_dir, date)):
        print(f'{data_dir} does not exist!')
        return False
    
    # Check for the existence of index and multistream data files
    fname_base = f'enwiki-{date.replace("-", "")}-pages-articles-multistream'
    for fname_suffix in {'.xml', '-index.txt'}:
        if not path.exists(fpath:=path.join(dump_dir, f'{fname_base}{fname_suffix}.bz2')):
            print(f'{fpath} does not exist!')
            return False
        
    # Files exist
    return True

# Validate 
validate_dump(DATA_DIR)

True

# Experimentation

In [ ]:
# Loading index for files
id2title = {}
id2offset = {}
fpath = path.join(DATA_DIR, '2021-10-20', 'enwiki-20211020-pages-articles-multistream-index.txt.bz2')
with bz2.open(fpath, 'rt', encoding='utf-8') as index_file:
    for line in index_file.readlines():
        print(line)
        break